Based on Notebook version: 29 Jan 2021 
(added small updates ~15 Feb)

In [ ]:
1 # just to check whether the kernel finished loading...

<a class="anchor" id="top"></a>
# Tutorial: scATAC-seq data analysis with pycisTopic

* *Index:*
    - [1. Getting pseudobulk profiles from scRNA-seq annotations](#pseudobulk)
    - [2. Inferring consensus peaks](#consensus_peaks)
    - [3. QC](#qc)
        - [3a. Sample-level statistics](#stats_sample)
        - [3b. Barcode level statistics](#stats_bc)
    - [4. Creating a cisTopic object](#create_object)
    - [5. Adding metadata to a cisTopic object](#add_metadata)
    - [6. Running scrublet in a cisTopic object](#scrublet)
    - [7. Run models](#run_models)
    - [8. Model selection](#model_selection)
    - [9. Clustering and visualization](#clustering)
    - [10. Topic binarization](#topic_bin)
    - [11. Differentially Accessible Regions (DARs)](#dars)
    - [12. Exporting to loom](#loom_export)


In [ ]:
# (Use your own copy from github to avoid issues)
pycisTopicPath = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/pycisTopic/'
#pycisTopicPath = '/staging/leuven/stg_00002/lcb/saibar/Projects/SCENIC_V2/packages/pycisTopic/'

In [ ]:
%matplotlib inline
import os
os.chdir(pycisTopicPath)
from pycisTopic.cistopic_class import *
from pycisTopic.qc import *
from pycisTopic.lda_models import *
from pycisTopic.clust_vis import *
from pycisTopic.topic_filtering import *
from pycisTopic.diff_features import *
from pycisTopic.pseudobulk_peak_calling import *
from pycisTopic.iterative_peak_calling import *
from pycisTopic.topic_binarization import *
from pycisTopic.export_to_loom import *
import pickle 

In [ ]:
# collected all imports to check dependencies (also kept in corresponding fields):
import pyranges as pr
import requests
import pybiomart as pbm

Set paths and data files:

In [ ]:
# Output path for this analysis:
outDir = '/staging/leuven/stg_00002/lcb/saibar/Projects/SCENIC_V2/10x_multiome_brain/' + 'output/atac/pycistopic/'

# Data location:
datasetRootPath = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/10x_multiome_brain/'
path_to_annotated_rna_loom = datasetRootPath + 'output/rna/vsn/add_clusters_scrublet_as_annot/10x_multiome_brain_SCENIC_SCope_output_wAnnot_noDBL.loom'
path_to_annotated_rna_loom_withDoublets = datasetRootPath + 'output/rna/vsn/add_clusters_scrublet_as_annot/10x_multiome_brain_SCENIC_SCope_output-wAnnot-wDBL.loom'
fragments_dict = {'10x_multiome_brain': datasetRootPath + '/data/cell-arc/1.0.0/outs/human_brain_3k_atac_fragments.tsv.gz'}

# Settings/databases, etc...:
path_to_blacklist = pycisTopicPath + 'blacklist/hg38-blacklist.v2.bed'

In [ ]:
if not os.path.exists(outDir):
    os.makedirs(outDir)
os.listdir(outDir)

<a class="anchor" id="pseudobulk"></a>
### 1. Getting pseudobulk profiles from scRNA-seq annotations 

We will start by loading the barcode metadata from the scRNA-seq analysis. In this case, I will use the annotated loom file, but loading it from a tsv file or similar is also possible (as long as you end up with a similar pd.DataFrame).

In [ ]:
# Create function to get metadata from loom (to add to SCopeLoomPy)
def get_metadata(loom):
    annot_metadata = loom.get_meta_data()['annotations']
    annot_mt_column_names = [annot_metadata[x]['name'] for x in range(len(annot_metadata))]
    annot_mt = pd.concat([pd.DataFrame(loom.col_attrs[annot_mt_column_names[x]]) for x in range(len(annot_mt_column_names))], axis=1)
    annot_mt.columns = [annot_mt_column_names[x] for x in range(len(annot_mt_column_names))]
    annot_mt.index = loom.get_cell_ids().tolist()
    return annot_mt

In [ ]:
# Get metadata from loom file
loom = SCopeLoom.read_loom(path_to_annotated_rna_loom)
cell_data = get_metadata(loom)

Importantly, if you are starting from a pandas data frame, the index of your pandas should correspond to BARCODE (e.g. ATGTCTGATAGA-1, additional tags are possible using -; e.g. ATGTCTGATAGA-1-sample_1) and it must contain a 'sample_id' column indicating the sample label fo origin. Let's see how it looks like here. The sample_id for all cells in this tutorial is '10x_multiome_brain'. Alternatively, you can also add a column named 'barcode' to the metadata with the corresponding cell barcodes (in this case the name of the cells will not be used to infer the barcode id).

In [ ]:
cell_data

In order to produce the bigwig files, we also need to know the overall size of the chromosomes. We can easily download this information from the UCSC.

In [ ]:
# Get chromosome sizes (for hg38 here)
import pyranges as pr
import requests
chromSizes_url='http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes'
chromsizes=pd.read_csv(chromSizes_url, sep='\t', header=None)
chromsizes.columns=['Chromosome', 'End']
chromsizes['Start']=[0]*chromsizes.shape[0]
chromsizes=chromsizes.loc[:,['Chromosome', 'Start', 'End']]
# Exceptionally in this case, to agree with CellRangerARC annotations
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].replace('v', '.') for x in range(len(chromsizes['Chromosome']))]
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].split('_')[1] if len(chromsizes['Chromosome'][x].split('_')) > 1 else chromsizes['Chromosome'][x] for x in range(len(chromsizes['Chromosome']))]
chromsizes=pr.PyRanges(chromsizes)

Now we have all the ingredients we need  to generate the pseudobulk files. With this function we will generate fragments files per group and the corresponding bigwigs. The mandatory input to this function are:
* The annotation dataframe (`input_data`)
* The variable used to group the cells (in this case, 'cell type')
* The chromosome sizes
* The paths to where the bed and bigiwg files will be written
* A dictionary indicating the fragments file corresponsing to each sample. **The sample ids used as keys in this dictionary must match with the sample ids in the annotation data frame!**

The output will be two dictionaries containing the paths to the bed and bigwig files, respectively, to each group.

In [ ]:
bw_paths, bed_paths = export_pseudobulk(input_data = cell_data,
                 variable = 'cell_type',
                 chromsizes = chromsizes,
                 bed_path = outDir + 'consensus_peak_calling/pseudobulk_bed_files/',
                 bigwig_path =  outDir + 'consensus_peak_calling/pseudobulk_bw_files/',
                 path_to_fragments = fragments_dict,
                 n_cpu = 5,
                 normalize_bigwig = True,
                 remove_duplicates = True)

In [ ]:
os.listdir(outDir+"consensus_peak_calling/pseudobulk_bed_files")

In [ ]:
os.listdir(outDir+'consensus_peak_calling/pseudobulk_bw_files')

Let's save the paths dictionaries.

In [ ]:
with open(outDir + 'consensus_peak_calling/pseudobulk_bed_files/bed_paths.pkl', 'wb') as f:
  pickle.dump(bed_paths, f)

In [ ]:
with open(outDir + 'consensus_peak_calling/pseudobulk_bed_files/bw_paths.pkl', 'wb') as f:
  pickle.dump(bw_paths, f)

This function can be used with cisTopic objects (as `input_data`), instead of the annotation data frame and the paths to fragments dictionary. You can find an example later.

<a class="anchor" id="consensus_peaks"></a>
### 2. Inferring consensus peaks

In the following step, we will use MACS2 to call peaks in each group (in this case, cell type). The default parameters are those recommended for ATAC-seq data.

In [ ]:
import os
# Set correct path to run MACS2
os.putenv('MACS2_HOME','/data/leuven/software/biomed/haswell_centos7/2018a/software/MACS2/2.1.2.1-foss-2018a-Python-2.7.16/')
os.putenv('PATH','/data/leuven/software/biomed/haswell_centos7/2018a/software/MACS2/2.1.2.1-foss-2018a-Python-2.7.16/bin:' + os.environ['PATH'])
macs_path =' /data/leuven/software/biomed/haswell_centos7/2018a/software/MACS2/2.1.2.1-foss-2018a-Python-2.7.16/bin/macs2'
macs_outdir = outDir + 'consensus_peak_calling/MACS/'

# Run peak calling
narrow_peaks_dict = peak_calling(macs_path, bed_paths, macs_outdir, n_cpu=5,
                    genome_size='hs', input_format='BEDPE', shift=73, ext_size=146, keep_dup = 'all', q_value = 0.05)

Let's save the narrow peaks dictionary (with a PyRanges with the narrow peaks for each cell type).

In [ ]:
with open(macs_outdir + 'narrow_peaks_dict.pkl', 'wb') as f:
  pickle.dump(narrow_peaks_dict, f)

Finally, it is time to derive the consensus peaks. To do so, we use the TGCA iterative peak filtering approach. First, each summit is extended a `peak_half_width` in each direction and then we iteratively filter out less significant peaks that overlap with a more significant one. During this procedure peaks will be merged and depending on the number of peaks included into them, different processes will happen:
* **1 peak**:  The original peak region will be kept
* **2 peaks**:  The original peak region with the highest score will be kept
* **3 or more peaks**:  The orignal peak region with the most significant score will be taken, and all the original peak regions in this merged peak region that overlap with the significant peak region will be removed. The process is repeated with the next most significant peak (if it was not removed already) until all peaks are processed.

This proccess will happen twice, first in each pseudobulk peaks; and after peak score normalization, to process all peaks together.

In [ ]:
# Get chromosome sizes (for hg38 here). We need them to ensure that extending the summits we don't fall out of the chromosome.
import pyranges as pr
import requests
chromSizes_url='http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes'
chromsizes=pd.read_csv(chromSizes_url, sep='\t', header=None)
chromsizes.columns=['Chromosome', 'End']
chromsizes['Start']=[0]*chromsizes.shape[0]
chromsizes=chromsizes.loc[:,['Chromosome', 'Start', 'End']]
# Exceptionally in this case, to agree with CellRangerARC annotations
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].replace('v', '.') for x in range(len(chromsizes['Chromosome']))]
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].split('_')[1] if len(chromsizes['Chromosome'][x].split('_')) > 1 else chromsizes['Chromosome'][x] for x in range(len(chromsizes['Chromosome']))]
chromsizes=pr.PyRanges(chromsizes)

In [ ]:
# Other param
peak_half_width=250
# Get consensus peaks
consensus_peaks=get_consensus_peaks(narrow_peaks_dict, peak_half_width, chromsizes=chromsizes, path_to_blacklist=path_to_blacklist) 

In [ ]:
# Write to bed
consensus_peaks.to_bed(path=outDir + 'consensus_peak_calling/consensus_regions.bed', keep=True, compression='infer', chain=False)

<a class="anchor" id="qc"></a>
### 3. QC

The next step is to perform QC in the scATAC-seq samples (in this case, only one run). There are several measurements and visualizations performed in this step:

* Barcode rank plot
* Duplication rate
* Insertion size
* TSS enrichment
* Fraction of Reads In Peaks (FRIP)

To calculate the TSS enrichment we need to provide TSS annotations. You can easily download them via pybiomart.

In [ ]:
# Get TSS annotations
import pybiomart as pbm
# For mouse
#dataset = pbm.Dataset(name='mmusculus_gene_ensembl',  host='http://www.ensembl.org')
# For human
dataset = pbm.Dataset(name='hsapiens_gene_ensembl',  host='http://www.ensembl.org')
# For fly
#dataset = pbm.Dataset(name='dmelanogaster_gene_ensembl',  host='http://www.ensembl.org')
annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
filter = annot['Chromosome/scaffold name'].str.contains('CHR|GL|JH|MT')
annot = annot[~filter]
annot['Chromosome/scaffold name'] = annot['Chromosome/scaffold name'].str.replace(r'(\b\S)', r'chr\1')
annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
annot = annot[annot.Transcript_type == 'protein_coding']

If you want to run all (or several of) the metrics, you can use the `compute_qc_stats()` function. As input you need to provide a dictionary containing the fragments files per sample and another dictionary the corresponding regions to use to estimate the FRIP. For more details in the QC stats, see the *QC* advanced tutorial.

In [ ]:
path_to_regions_qc = {'10x_multiome_brain': outDir + 'consensus_peak_calling/consensus_regions.bed'}

metadata_bc, profile_data_dict = compute_qc_stats(fragments_dict = fragments_dict,
                tss_annotation = annot,
                stats=['barcode_rank_plot', 'duplicate_rate', 'insert_size_distribution', 'profile_tss', 'frip'],
                label_list = None,
                path_to_regions = path_to_regions_qc,
                n_cpu = 5,
                valid_bc = None,
                n_frag = 100,
                n_bc = None,
                tss_flank_window = 1000,
                tss_window = 50,
                tss_minimum_signal_window = 100,
                tss_rolling_window = 10,
                remove_duplicates = True)

In [ ]:
if not os.path.exists(outDir + 'quality_control'):
    os.makedirs(outDir + 'quality_control')

In [ ]:
with open(outDir + 'quality_control/metadata_bc.pkl', 'wb') as f:
  pickle.dump(metadata_bc, f)

In [ ]:
with open(outDir + 'quality_control/profile_data_dict.pkl', 'wb') as f:
  pickle.dump(profile_data_dict, f)

<a class="anchor" id="stats_sample"></a>
#### 3a. Sample-level statistics

Once the QC metrics have been computed you can visualize the results at the sample-level and the barcode-level. Sample-level statistics can be used to assess the overall quality of the sample, while barcode level statistics can be use to differentiate good quality cells versus the rest. The sample-level graphs include:

* **Barcode rank plot**: The barcode rank plot shows the distribution of non-duplicate reads and which barcodes were inferred to be associated with cells. A steep drop-off ('knee') is indicative of good separation between the cell-associated barcodes and the barcodes associated with empty partitions.

* **Insertion size**: ATAC-seq requires a proper pair of Tn5 transposase cutting events at the ends of DNA. In the nucleosome-free open chromatin regions, many molecules of Tn5 can kick in and chop the DNA into small pieces; around nucleosome-occupied regions, and Tn5 can only access the linker regions. Therefore, in a good ATAC-seq library, you should expect to see a sharp peak at the <100 bp region (open chromatin), and a peak at ~200bp region (mono-nucleosome), and other larger peaks (multi-nucleosomes). A clear nucleosome pattern indicates a good quality of the experiment.

* **Sample TSS enrichment**: The TSS enrichment calculation is a signal to noise calculation. The reads around a reference set of TSSs are collected to form an aggregate distribution of reads centered on the TSSs and extending to 1000 bp in either direction (for a total of 2000bp). This distribution is then normalized by taking the average read depth in the 100 bps at each of the end flanks of the distribution (for a total of 200bp of averaged data) and calculating a fold change at each position over that average read depth. This means that the flanks should start at 1, and if there is high read signal at transcription start sites (highly open regions of the genome) there should be an increase in signal up to a peak in the middle.

* **FRIP distribution**: Fraction of all mapped reads that fall into the called peak regions, i.e. usable reads in significantly enriched peaks divided by all usable reads. In general, FRiP scores correlate positively with the number of regions. A low FRIP indicates that many reads form part of the background, and so that the data is noisy.

* **Duplication rate**: A fragment is considered “usable” if it uniquely maps to the genome and remains after removing PCR duplicates (defined as two fragments that map to the same genomic position and have the same unique molecular identifier). The duplication rate serves to estimate the amount of usable reads per barcode. High duplication rates may indicate over-sequencing or lack of fragments after transposition and encapsulation.

In [ ]:
plot_sample_metrics(profile_data_dict,
           insert_size_distriubtion_xlim=[0,600],
           ncol=5,
           save=outDir + 'quality_control/sample_metrics.pdf')

<a class="anchor" id="stats_bc"></a>
#### 3b. Barcode level statistics

Barcode-level statistics can be used to select high quality cells. Typical measurements that can be used are:

* **Total number of (unique) fragments**
* **TSS enrichment**: The score at position in the TSS enrichmen score for for each barcode (at position 0, the TSS). Noisy cells will have a low TSS enrichment.
* **FRIP**: The fraction of reads in peaks for each barcode. Noisy cells have low FRIP values. However, this filter should be used with nuance, as it depends on the quality of the original peaks. For example, if there is a rare population in the sample, its specific peaks may be missed by peak calling algorithms, causing a decrease in their FRIP values.

In [ ]:
# # Load barcode metrics
# import pickle
# infile = open(outDir + 'quality_control/metadata_bc.pkl', 'rb')
# metadata_bc = pickle.load(infile)
# infile.close()

In [ ]:
# TODO: include in plot_barcode_metrics?
if not os.path.exists(outDir + 'quality_control/barcode_metrics'):
    os.makedirs(outDir + 'quality_control/barcode_metrics')

In [ ]:
# Return figure to plot together with other metrics, and cells passing filters. Figure will be saved as pdf.
FRIP_NR_FRAG_fig, FRIP_NR_FRAG_filter = plot_barcode_metrics(metadata_bc['10x_multiome_brain'],
                                       var_x='Log_unique_nr_frag',
                                       var_y='FRIP',
                                       min_x=3.5,
                                       max_x=None,
                                       min_y=20,
                                       max_y=None,
                                       return_cells=True,
                                       return_fig=True,
                                       plot=False,
                                       save=outDir + 'quality_control/barcode_metrics/FRIP-VS-NRFRAG.pdf')
# Return figure to plot together with other metrics, and cells passing filters
TSS_NR_FRAG_fig, TSS_NR_FRAG_filter=plot_barcode_metrics(metadata_bc['10x_multiome_brain'],
                                      var_x='Log_unique_nr_frag',
                                      var_y='TSS_enrichment',
                                      min_x=3.5,
                                      max_x=None,
                                      min_y=5,
                                      max_y=None,
                                      return_cells=True,
                                      return_fig=True,
                                      plot=False,
                                      save=outDir + 'quality_control/barcode_metrics/TSS-VS-NRFRAG.pdf')
# Return figure to plot together with other metrics, but not returning cells (no filter applied for the duplication rate  per barcode)
DR_NR_FRAG_fig=plot_barcode_metrics(metadata_bc['10x_multiome_brain'],
                                      var_x='Log_unique_nr_frag',
                                      var_y='Dupl_rate',
                                      min_x=3.5,
                                      max_x=None,
                                      min_y=5,
                                      max_y=None,
                                      return_cells=False,
                                      return_fig=True,
                                      plot=False)

# Plot barcode stats in one figure
fig=plt.figure(figsize=(40,10))
plt.subplot(1, 3, 1)
img = fig2img(FRIP_NR_FRAG_fig) #To convert figures to png to plot together, see .utils.py. This converts the figure to png.
plt.imshow(img)
plt.axis('off')
plt.subplot(1, 3, 2)
img = fig2img(TSS_NR_FRAG_fig)
plt.imshow(img)
plt.axis('off')
plt.subplot(1, 3, 3)
img = fig2img(DR_NR_FRAG_fig)
plt.imshow(img)
plt.axis('off')
plt.show()

In this case, we have information about doublet barcodes from the scRNA-seq analysis. We will remove these barcodes from our selected barcodes.

In [ ]:
# Remove cells marked as doublets in the scRNA-seq analysis
loom = SCopeLoom.read_loom(path_to_annotated_rna_loom_withDoublets)
cell_data = get_metadata(loom)
# Here I apply a corection to get the correct barcode name
SCRUBLET_doublets = [re.sub("-1.0.0", "-1", x) for x in cell_data[cell_data['scrublet__predicted_doublets'] == 1].index.tolist()]

In [ ]:
bc_passing_filters = {'10x_multiome_brain':[]}
bc_passing_filters['10x_multiome_brain'] = list((set(FRIP_NR_FRAG_filter) & set(TSS_NR_FRAG_filter)) ^ set(SCRUBLET_doublets))

We have a total of 3,154 selected barcodes.

In [ ]:
len(bc_passing_filters['10x_multiome_brain'])

Of these, a total of 2,419 overlaps with high quality scRNA-seq barcodes. We will keep the additional barcodes.

In [ ]:
# Get metadata from high-quality loom file
loom = SCopeLoom.read_loom(path_to_annotated_rna_loom)
cell_data = get_metadata(loom)
scRNA_bc=[re.sub("-10x_multiome_brain", "", x) for x in cell_data.index.tolist()]
#len(scRNA_bc) 2,607 cells
len(list(set(bc_passing_filters['10x_multiome_brain']) & set(scRNA_bc)))

In [ ]:
with open(outDir + 'quality_control/bc_passing_filters.pkl', 'wb') as f:
  pickle.dump(bc_passing_filters, f)

<a class="anchor" id="create_object"></a>
### 4. Creating a cisTopic object

This step will generate a **count matrix** with the fragments in each region for each barcode. 
If you would like to start from a precomputed fragments count matrix it is also possible (see advanced tutorial on *Initializing cisTopic objects*). 
For multiple samples, you can add additional entries in `fragment_dict`. 
As regions, we will use the consensus peaks derived from the scRNA-seq annotations. 

This cisTopic object will contain:

* Path/s to fragment file/s (if generated from fragments files)
* Fragment count matrix and binary accessibility matrix
* Cell and region metadata

In [ ]:
path_to_regions_ct = outDir + 'consensus_peak_calling/consensus_regions.bed'

In [ ]:
# Metrics
infile = open(outDir + 'quality_control/metadata_bc.pkl', 'rb')
metadata_bc = pickle.load(infile)
infile.close()
# Valid barcodes
infile = open(outDir + 'quality_control/bc_passing_filters.pkl', 'rb')
bc_passing_filters = pickle.load(infile)
infile.close()
#Create objects
cistopic_obj_list=[create_cistopic_object_from_fragments(path_to_fragments=fragments_dict[key],
                                               path_to_regions=path_to_regions_ct,
                                               path_to_blacklist=path_to_blacklist,
                                               metrics=metadata_bc[key],
                                               valid_bc=bc_passing_filters[key],
                                               n_cpu=5,
                                               project=key) for key in fragments_dict.keys()]

In this case we only have one sample, so only one cisTopic object has been generated. If you would have multiple samples, you would need to run the `merge()` function in your cisTopic object list. You can find more information in the advanced tutorial on *Initializing cisTopic objects*.

In [ ]:
cistopic_obj = cistopic_obj_list[0]
print(cistopic_obj)

In [ ]:
# Save
with open(outDir + '10x_multiome_brain_cisTopicObject.pkl', 'wb') as f:
  pickle.dump(cistopic_obj, f)

<a class="anchor" id="add_metadata"></a>
### 5. Adding metadata to a cisTopic object

We can add additional metadata (for regions or cells) to a cisTopic object. For example, let's add the scRNA-seq data annotations. For those barcodes that did not pass the scRNA-seq values will be filled with `Nan`.

In [ ]:
# Load cisTopic object
infile = open(outDir + '10x_multiome_brain_cisTopicObject.pkl', 'rb')
cistopic_obj = pickle.load(infile)
infile.close()

In [ ]:
# Create function to get metadata from loom (to add to SCopeLoomPy)
def get_metadata(loom):
    annot_metadata = loom.get_meta_data()['annotations']
    annot_mt_column_names = [annot_metadata[x]['name'] for x in range(len(annot_metadata))]
    annot_mt = pd.concat([pd.DataFrame(loom.col_attrs[annot_mt_column_names[x]]) for x in range(len(annot_mt_column_names))], axis=1)
    annot_mt.columns = [annot_mt_column_names[x] for x in range(len(annot_mt_column_names))]
    annot_mt.index = loom.get_cell_ids().tolist()
    return annot_mt

In [ ]:
# Get metadata from loom file
loom = SCopeLoom.read_loom(path_to_annotated_rna_loom)
cell_data = get_metadata(loom).drop('sample_id', axis=1) 

The indexes in the pandas data frame to add can be **cell barcodes** (if the cisTopic object has been created from a fragments file only) or an **exact match with the cell names** in the cisTopic object (`cistopic_obj.cell_names`). 

In [ ]:
cistopic_obj.add_cell_data(cell_data)

In [ ]:
cistopic_obj.cell_data

<a class="anchor" id="scrublet"></a>
### 6. Running scrublet in a cisTopic object

Optionally, you can run also scrublet on the fragment count matrix to infer doublets from the scATAC-seq. 

In [ ]:
import scrublet as scr
scrub = scr.Scrublet(cistopic_obj.fragment_matrix.T, expected_doublet_rate=0.1)
doublet_scores, predicted_doublets = scrub.scrub_doublets()
scrub.plot_histogram();
scrub.call_doublets(threshold=0.35)
scrub.plot_histogram();
scrublet = pd.DataFrame([scrub.doublet_scores_obs_, scrub.predicted_doublets_], columns=cistopic_obj.cell_names, index=['Doublet_scores_fragments', 'Predicted_doublets_fragments']).T

In [ ]:
cistopic_obj.add_cell_data(scrublet)

141 cells are called as doublets. We will keep them to see how the look in the analysis, but you can already remove them.

In [ ]:
sum(cistopic_obj.cell_data.Predicted_doublets_fragments == True)

In [ ]:
# Save with doublets
with open(outDir + '10x_multiome_brain_cisTopicObject.pkl', 'wb') as f:
  pickle.dump(cistopic_obj, f)

We can subset all cells marked as singlets from the cisTopic object.

In [ ]:
# Remove doublets 
singlets = cistopic_obj.cell_data[cistopic_obj.cell_data.Predicted_doublets_fragments == False].index.tolist()
# Subset cisTopic object
cistopic_obj_noDBL = cistopic_obj.subset(singlets, copy=True)
print(cistopic_obj_noDBL)

In [ ]:
# Save without doublets
with open(outDir + '10x_multiome_brain_cisTopicObject_noDBL.pkl', 'wb') as f:
  pickle.dump(cistopic_obj_noDBL, f)

### 7. Run models

The next step is to run the LDA models. There are two types of LDA models (with Collapsed Gibbs Sampling) you can run:

* **Serial LDA**: The parallelization is done between models rather than within each model. Recommended for small-medium sized data sets in which several models with different number os topics are being tested. You can run these models with `runCGSModels()`.
* **Parallel LDA with MALLET**: The parallelization is done within each model. Recommended for large data sets where a few models with different number of topics are being tested. If working in a cluster, we recommed to submit a job per model so they can run simultaneously. You can run it with `runCGSModelsMallet()`.

In this tutorial we will use `runCGSModels`. For more details in how to run models, see the advanced tutorial on *Running LDA models*.

In [ ]:
models=run_cgs_models(cistopic_obj,
                    n_topics=[2,10,20,30,40,50],
                    n_cpu=6,
                    n_iter=100,
                    random_state=555,
                    alpha=50,
                    alpha_by_topic=True,
                    eta=0.1,
                    eta_by_topic=False,
                    save_path=None)

To add models with other numbers of topics (e.g. run at a later time...): 

In [ ]:
models_to_add=run_cgs_models(cistopic_obj,
                    n_topics=[5,15,25,35,45],
                    n_cpu=6,
                    n_iter=100,
                    random_state=555,
                    alpha=50,
                    alpha_by_topic=True,
                    eta=0.1,
                    eta_by_topic=False,
                    save_path=None)
models = models + models_to_add

In [ ]:
# Save
if not os.path.exists(outDir + 'models/'):
    os.makedirs(outDir + 'models/')
with open(outDir + 'models/10x_multiome_brain_models_100_iter.pkl', 'wb') as f:
  pickle.dump(models, f)

If you are working on a cluster and want to run several models, we recommend to submit this step as a job. You can use the `runModels_lda.py` script.

<a class="anchor" id="model_selection"></a>
### 8. Model selection

There are several methods that can be used for model selection:

* **Minmo_2011**: Uses the average model coherence as calculated by Mimno et al (2011). In order to reduce the impact of the number of topics, we calculate the average coherence based on the top selected average values. The better the model, the higher coherence.
* **Log-likelihood**: Uses the log-likelihood in the last iteration as calculated by Griffiths and Steyvers (2004). The better the model, the higher the log-likelihood.
* **Arun_2010**: Uses a density-based metric as in Arun et al (2010) using the topic-region distribution, the cell-topic distribution and the cell coverage. The better the model, the lower the metric.
* **Cao_Juan_2009**: Uses a divergence-based metric as in Cao Juan et al (2009) using the topic-region distribution. The better the model, the lower the metric.

For scATAC-seq data models, the most helpful methods are Minmo (topic coherence) and the log-likelihood in the last iteration.

In [ ]:
# # Load models
# infile = open(outDir + 'models/10x_multiome_brain_models_100_iter.pkl', 'rb')
# models = pickle.load(infile)
# infile.close()

In [ ]:
# order:
models=[models[i] for i in np.argsort([m.n_topic for m in models])]
[m.n_topic for m in models]

In [ ]:
model=evaluate_models(models,
                     select_model=25, 
                     return_model=True, 
                     metrics=['Arun_2010','Cao_Juan_2009', 'Minmo_2011', 'loglikelihood'],
                     plot_metrics=False)

In [ ]:
# Load cisTopic object
infile = open(outDir + '10x_multiome_brain_cisTopicObject.pkl', 'rb')
cistopic_obj = pickle.load(infile)
infile.close()
# Add model to cisTopicObject
cistopic_obj.add_LDA_model(model)

In [ ]:
# Save
with open(outDir + '10x_multiome_brain_cisTopicObject.pkl', 'wb') as f:
  pickle.dump(cistopic_obj, f)

<a class="anchor" id="clustering"></a>
### 9. Clustering and visualization

We can cluster the cells (or regions) using the leiden algorithm, and perfor dimensionality reductiion with UMAP and TSNE. In these examples we will focus on the cells only. For these steps, the cell-topic contriibutions of the model will be used.

In [ ]:
find_clusters(cistopic_obj,
                 target  = 'cell',
                 k = 10,
                 res = 0.6,
                 prefix = 'pycisTopic_')

In [ ]:
run_umap(cistopic_obj, target  = 'cell')

In [ ]:
run_tsne(cistopic_obj, target  = 'cell')

The clustering assignments are added to `cistopic_obj.cell_data` and the projections to `cistopic_obj.projections['cell']`. If you would like to add additional dimensionality reductions, you can just add them as an entry to the projections dictionary (under 'cell' in this case).

In [ ]:
cistopic_obj.cell_data

We can also visualize metadata (categorical or continuous) on the UMAP/tSNE spaces.

In [ ]:
# TODO: include in plot_metadata?
if not os.path.exists(outDir + 'visualization'):
    os.makedirs(outDir + 'visualization')

In [ ]:
plot_metadata(cistopic_obj,
                 reduction_name='UMAP',
                 variables=['cell_type', 'pycisTopic_leiden_10_0.6'], # Labels from RNA and new clusters
                 target='cell', num_columns=2,
                 text_size=24,
                 dot_size=15,
                 figsize=(25,10),
                 save=outDir + 'visualization/dimensionality_reduction_label.pdf')

We can also check now where our predicted doublets fall. Many of these predicted doublets actually correspond to cells with no high quality RNA profile.

In [ ]:
plot_metadata(cistopic_obj,
                 reduction_name='UMAP',
                 variables=['Log_unique_nr_frag', 'TSS_enrichment', 'Doublet_scores_fragments', 'Predicted_doublets_fragments'], 
                 target='cell', num_columns=4,
                 text_size=15,
                 save=outDir + 'visualization/dimensionality_reduction_number.pdf')

We can also plot the topic-contributions.

In [ ]:
plot_topic(cistopic_obj,
            reduction_name = 'UMAP',
            target = 'cell',
            num_columns=4,
            save=outDir + 'visualization/dimensionality_reduction_topic_contr.pdf')

Or we can also draw a heatmap with the topic contributions (and annotations).

In [ ]:
cell_topic_heatmap(cistopic_obj,
                     variables = ['cell_type'],
                     scale = True,
                     legend_loc_x = 1.05,
                     legend_loc_y = -1.2,
                     legend_dist_y = -1,
                     figsize=(10,10),
                     save = None)

<a class="anchor" id="topic_bin"></a>
### 10. Topic binarization

<a class="anchor" id="dars"></a>
### 11. Differentially Accessible Regions (DARs)

<a class="anchor" id="loom_export"></a>
### 12. Exporting to loom

[[Back to top]](#top)